In [ ]:
#1-import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob as gb
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
print("Packages imported...")

In [ ]:
#2-import dataset
data='D:/Downloads/1-MSA/ML/Project/New folder (2)/asl_alphabet_train'

In [ ]:
for folder in  os.listdir(data) : 
    pathname= str( data +'/'+ folder)
    files = gb.glob(pathname)
    print(f'For training data , found {len(files)} in folder {folder}')
    print(pathname)

In [ ]:
#dictionary of all exist classes and its numberic values in used in dataset 
code = {'A':0 ,'B':1,'C':2,'D':3,'E':4,'F':5,'G': 6,'H':7 ,'I':8 ,'J':9,
        'K':10,'L':11,'M':12, 'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,
        'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,'Z':25,'del':26,'nothing':27,'space':28}

#used for getting name of class x  using its number n or y
def getcode(n) : 
    for x , y in code.items() : 
        if n == y : 
            return x
print("done")
s=128

In [ ]:
#empty list to contain images itself called featues for the CNN model  
X_data = []
#empty list to contain actual value for each image
y_data = []
for folder in  os.listdir(data) : 
    pathname= str( data +'/'+ folder)
    files = gb.glob(os.path.join(pathname, '*'))
    for file in files: 
        image = cv2.imread(file)
        #using cv2.resize without determine interpolation make it preserve aspect ratio for each image 
        image_array = cv2.resize(image , (s,s))
        X_data.append(list(image_array))
        y_data.append(code[folder])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2,random_state=42,stratify=y_data)

In [ ]:
model =Sequential()

model.add(Conv2D(120, (5, 5), input_shape=(s, s, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(100, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(80, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(50, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(29, activation='softmax'))

model.summary()


In [ ]:
# One-Hot-Encoding the categorical data
y_cat_train = to_categorical(y_train,29)
y_cat_test = to_categorical(y_test,29)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
#X_pred_array = np.array(X_pred)
y_train = np.array(y_train)
y_test = np.array(y_test)

print(f'X_train shape  is {X_train.shape}')
print(f'X_test shape  is {X_test.shape}')
#print(f'X_pred shape  is {X_pred_array.shape}')
print(f'y_train shape  is {y_train.shape}')
print(f'y_test shape  is {y_test.shape}')

In [ ]:
model.fit(X_train, y_cat_train,epochs=50,batch_size=64,validation_data=(X_test, y_cat_test),callbacks=[early_stop])